In [1]:
# Import objects from pyomo package
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           Constraint,
                           Expression,
                           TransformationFactory,
                           value,
                           units as pyunits)

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Todo: import Flash unit model from idaes.generic_models.unit_models
from idaes.generic_models.unit_models import Flash

In [2]:
from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

import pyomo.contrib.parmest.parmest as parmest

In [3]:
# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import Pandas
import pandas as pd

In [4]:
from R125_R32_PR3 import configuration

# Load data from csv
data = pd.read_csv('R32R125298.csv')

In [5]:
def PR_model(data):
    
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})

    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(298.15)
    m.fs.state_block.pressure.fix(1669700)
    m.fs.state_block.mole_frac_comp["R32"].fix(0.2688)
    m.fs.state_block.mole_frac_comp["R125"].fix(0.7312)
    
    m.fs.properties.PR_kappa['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa['R32', 'R125'].fix(0.004)
    m.fs.properties.PR_kappa['R125', 'R32'].fix(0.004)

    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix at actual temperature
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.temperature.fix(float(data["temperature"]))
#     m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.mole_frac_comp["R32"].fix(float(data["x_R32"]))
    m.fs.state_block.mole_frac_comp["R125"].unfix()
#     m.fs.state_block.mole_frac_comp["R32"].unfix()
#     m.fs.state_block.mole_frac_comp["R32"].fix(float(data["x_R32"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data["x_R125"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data["x_R32"]))
#     m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].unfix()
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].fix(float(data["y_R125"]))
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R32'].fix(float(data["y_R32"]))
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R32'].unfix()
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].unfix()
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].unfix()
    
#     m.fs.kappas = Constraint(expr=m.fs.properties.PR_kappa['R32', 'R125'] == 
#                     m.fs.properties.PR_kappa['R125', 'R32'])
#     m.fs.properties.PR_kappa['R32', 'R125'] = Expression(expr=value(m.fs.properties.PR_kappa['R125', 'R32']))
    
    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa['R32', 'R125'].setlb(-3)
    m.fs.properties.PR_kappa['R32', 'R125'].setub(3)

    m.fs.properties.PR_kappa['R125', 'R32'].setlb(-3)
    m.fs.properties.PR_kappa['R125', 'R32'].setub(3)

    # Return initialized flash model
    return m

In [6]:
import pytest

test_data = {"temperature": 298.15, "pressure": 1669700, "x_R32":0.7312, 
             "x_R125":0.2688, "y_R125":0.2501, "y_R32":0.7499}

m = PR_model(test_data)

DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

# Check that degrees of freedom is 0
# assert degrees_of_freedom(m) == 0

2021-06-08 14:09:13 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.
2021-06-08 14:09:15 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
The initial DOF is 0


In [7]:
variable_name = ["fs.properties.PR_kappa['R32', 'R125']",
                 "fs.properties.PR_kappa['R125', 'R32']"]

In [8]:
def SSE(m, data):
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
    return expr*1e-9

# def SSE(m, data):
#     expr = (
#             (float(data["y_R125"]) -
#              m.fs.state_block.mole_frac_phase_comp["Vap", "R125"])**2
# #             + (float(data["x_R125"]) -
# #              m.fs.state_block.mole_frac_phase_comp["Liq", "R125"])**2
#            )
    
#     return expr*1e4


In [9]:
solver_options = {'tol': 1e-4}
pest = parmest.Estimator(PR_model, data, variable_name, SSE, solver_options)
# solver_options = {'tol': 1e-6, 'max_iter':100}

obj_value, parameters = pest.theta_est()

2021-06-08 14:09:15 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.
2021-06-08 14:09:17 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-06-08 14:09:17 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.
2021-06-08 14:09:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-06-08 14:09:19 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.
2021-06-08 14:09:21 [INFO] idaes.init.fs.state_

2021-06-08 14:10:08 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.
2021-06-08 14:10:12 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-06-08 14:10:12 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.
2021-06-08 14:10:14 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-06-08 14:10:14 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.
2021-06-08 14:10:17 [INFO] idaes.init.fs.state_

2021-06-08 14:11:18 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.
2021-06-08 14:11:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-06-08 14:11:21 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.
2021-06-08 14:11:24 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-06-08 14:11:24 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.
2021-06-08 14:11:27 [INFO] idaes.init.fs.state_

   1  5.5395834e+02 1.10e+01 2.69e+05  -1.0 2.10e+08    -  3.14e-04 1.19e-03h  5
ERROR: Output stream closed before all output was written to it. The following
    was left in the output buffer:
    	'   1  5.5395834e+02 1.10e+01 2.69e+05  -1.0 2.10e+08    -  3.14e-04
    	1.19e-03h  5\n'
   2  5.5436462e+02 1.10e+01 2.69e+05  -1.0 2.04e+08    -  9.22e-03 1.11e-05h  7
ERROR: Output stream closed before all output was written to it. The following
    was left in the output buffer:
    	'   2  5.5436462e+02 1.10e+01 2.69e+05  -1.0 2.04e+08    -  9.22e-03
    	1.11e-05h  7\n'
   3r 5.5436462e+02 1.10e+01 9.99e+02   1.0 0.00e+00  -4.0 0.00e+00 3.78e-07R 12
   4r 5.5436471e+02 1.26e+01 1.15e+03   1.0 5.95e+03    -  6.61e-03 2.75e-03f  1
   5r 5.5436471e+02 1.18e+01 9.99e+02   1.0 1.05e+01   2.0 1.98e-01 6.45e-02f  1
   6r 5.5436471e+02 1.07e+01 1.18e+03   1.0 5.77e+00   3.3 7.33e-02 9.24e-02f  1
   7r 5.5436471e+02 1.06e+01 3.46e+03   1.0 3.59e+00   4.7 8.93e-03 1.41e-02h  1
   8r 5.5436471

  31  6.0584468e+02 3.69e+01 3.40e+09  -1.0 4.66e+07  -3.5 2.63e-02 2.61e-04h  6
ERROR: Output stream closed before all output was written to it. The following
    was left in the output buffer:
    	'  31  6.0584468e+02 3.69e+01 3.40e+09  -1.0 4.66e+07  -3.5 2.63e-02
    	2.61e-04h  6\n'
  32r 6.0584468e+02 3.69e+01 1.00e+03   1.6 0.00e+00  -4.0 0.00e+00 2.56e-07R 16
  33r 6.0584403e+02 1.96e+01 9.99e+02   1.6 1.61e+04    -  6.59e-04 1.97e-03f  1
ERROR: Output stream closed before all output was written to it. The following
    was left in the output buffer:
    	'  32r 6.0584468e+02 3.69e+01 1.00e+03   1.6 0.00e+00  -4.0 0.00e+00
    	2.56e-07R 16\n  33r 6.0584403e+02 1.96e+01 9.99e+02   1.6 1.61e+04    -
    	6.59e-04 1.97e-03f  1\n'
  34  6.0175950e+02 2.44e+01 6.42e+08  -1.0 2.29e+08    -  4.04e-04 1.04e-03F  1
ERROR: Output stream closed before all output was written to it. The following
    was left in the output buffer:
    	'  34  6.0175950e+02 2.44e+01 6.42e+08  -1.0 2.29e+08

...

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error in an AMPL evaluation. Run with "halt_on_ampl_

ERROR: Output stream closed before all output was written to it. The following
    was left in the output buffer:
    	'\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08\x08'


ValueError: Cannot load a SolverResults object with bad status: error

In [ ]:
print("The SSE at the optimal solution is %0.6f" % obj_value)
print()
print("The values for the parameters are as follows:")
for k,v in parameters.items():
    print(k, "=", v)

In [ ]:
pest.ef_instance.display()

In [ ]:
print(large_residuals_set(pest.ef_instance))